In [13]:
import tensorflow as tf
import numpy as np
T = 3  # Sequence length
D = 3   # Number of features

In [14]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
import glob

from RNN import RNN
#from CNN import CNN
from Transformer import Transformer

from utils import series_to_supervised


import pandas as pd
import matplotlib.pyplot as plt

df=pd.read_csv('AKBANK.csv')
df=df.reindex(columns=['Tarih','Şimdi','Açılış','Yüksek','Düşük','Hac.','Fark %'])
df.rename(columns = {'Tarih':'Date','Şimdi':'Now','Açılış':'Start','Yüksek':'Heigh','Düşük':'Low','Hac.':'Volume','Fark %':'Difference'}, inplace = True)
sl=['Now','Start','Heigh','Low']
df=df[sl]
df = df.applymap(lambda x: x.replace(',', '.'))
df=df.sort_index()

daily_data=df


look_back = 3
n_features = daily_data.shape[1]

X_train, y_train, X_test, y_test, scale_X = series_to_supervised(daily_data, train_size=0.8, n_in=look_back, n_out=1, target_column='Now', dropnan=True, scale_X=True)

X_train_reshaped = X_train.values.reshape((-1,look_back,n_features))
X_train_reshaped=X_train_reshaped[:, :,0:3]


X_test_reshaped = X_test.values.reshape((-1,look_back,n_features))
X_test_reshaped=X_test_reshaped[:, :,0:3]

y_train_reshaped = y_train.values.astype(float)
y_test_reshaped = y_test.values.astype(float)
print(X_test_reshaped.shape)


(4, 3, 3)


Score(t) = V * tanh(W1 * HT + W2 * prev_dec_hidden)

softmax(x) = exp(x) / sum(exp(x))

In [15]:
# Define the Attention layer
class Attention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(Attention, self).__init__()
        self.W = tf.keras.layers.Dense(units)#It must be equal to lstm layer unit number
 
        self.V = tf.keras.layers.Dense(1)#One output layer


    def call(self, inputs):
        # Compute attention weights
        score = tf.nn.relu(self.W(inputs))
        attention_weights = tf.nn.softmax(self.V(score), axis=1)

        context_vector = attention_weights * inputs
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector

# Build the model
def build_model(T, D):
    inputs = tf.keras.Input(shape=(T, D))
    x = tf.keras.layers.LSTM(128, return_sequences=True)(inputs)
    #x=tf.keras.layers.MultiHeadAttention(key_dim=128, num_heads=4, dropout=0.1)(x, x) Optional
    x = Attention(128)(x)
    x = tf.keras.layers.Dense(1)(x)
    model = tf.keras.Model(inputs=inputs, outputs=x)
    return model


# Build and compile the model
model = build_model(T, D)
model.compile(optimizer="adam", loss="mse")

# Train the model
model.fit(X_train_reshaped, y_train_reshaped, epochs=100, batch_size=32)
a=model.predict(X_test_reshaped)
y_test_reshaped


Epoch 1/100
1/1 [==============================] - 3s 3s/step - loss: 341.4639
Epoch 2/100
1/1 [==============================] - 0s 9ms/step - loss: 340.2405
Epoch 3/100
1/1 [==============================] - 0s 10ms/step - loss: 339.0099
Epoch 4/100
1/1 [==============================] - 0s 11ms/step - loss: 337.7675
Epoch 5/100
1/1 [==============================] - 0s 10ms/step - loss: 336.5101
Epoch 6/100
1/1 [==============================] - 0s 10ms/step - loss: 335.2310
Epoch 7/100
1/1 [==============================] - 0s 11ms/step - loss: 333.9285
Epoch 8/100
1/1 [==============================] - 0s 13ms/step - loss: 332.5930
Epoch 9/100
1/1 [==============================] - 0s 13ms/step - loss: 331.2154
Epoch 10/100
1/1 [==============================] - 0s 13ms/step - loss: 329.7879
Epoch 11/100
1/1 [==============================] - 0s 11ms/step - loss: 328.3002
Epoch 12/100
1/1 [==============================] - 0s 12ms/step - loss: 326.7407
Epoch 13/100
1/1 [==========

array([[16.34],
       [15.8 ],
       [16.37],
       [14.89]])

In [16]:
print('Real value for the last four day')
for i in range(len(a)):
    print('Prediction:',a[i],'Real Value:', y_test_reshaped[i],'\n')

Real value for the last four day
Prediction: [18.081861] Real Value: [16.34] 

Prediction: [17.874718] Real Value: [15.8] 

Prediction: [16.717216] Real Value: [16.37] 

Prediction: [14.078235] Real Value: [14.89] 

